In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
import requests
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
service = Service("/Users/tumendemberelshalkhaan/Downloads/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://www1.maine.gov/cgi-bin/online/mdoc/search-and-deposit/search.pl?Search=Continue')
ids = []

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
select = soup.find('select', {'name': 'location'})
options = select.find_all('option')
option_text = [option.text for option in options]
locations = [option for option in option_text if "Facility" in option]

In [ ]:
for location in locations:
    select = Select(driver.find_element("name", 'location'))
    select.select_by_visible_text(location)
    status = Select(driver.find_element("name", 'status'))
    status.select_by_visible_text('Incarcerated')
    driver.find_element("xpath", ".//*[@value='Search']").click()
    links = driver.find_elements("xpath", "//a[starts-with(@href, 'detail.pl?mdoc_number')]")
    for link in links:
        if link.text != "":
            ids.append(link.text)

    while True:
        try:
            next = driver.find_element("xpath", "//a[contains(text(), 'Next 30 results')]")
        except:
            break
        links = driver.find_elements("xpath", "//a[starts-with(@href, 'detail.pl?mdoc_number')]")
        for link in links:
            if link.text != "":
                ids.append(link.text)
        next.click()
    driver.get('https://www1.maine.gov/cgi-bin/online/mdoc/search-and-deposit/search.pl?Search=Continue')

In [ ]:
me = pd.DataFrame(columns=['ID', 'Name', 'Age', 'Race', 'Date Of Birth', 'Facility', 'Gender', 'Offense Description', 'Sentence Length'])

In [ ]:
for id in ids:    
    url = ('https://www1.maine.gov/cgi-bin/online/mdoc/search-and-deposit/detail.pl?mdoc_number1=' + id)
    response = requests.get(url)

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the HTML element that contains the offender's name

    id_elem = soup.find('td', text='MDOC Number:')
    id = id_elem.find_next_sibling('td').get_text()

    name_elem = soup.find('td', text='Last Name, First Name, Middle Initial:')
    name = name_elem.find_next_sibling('td').get_text()

    age_elem = soup.find('td', text='Age (Years):')
    age = age_elem.find_next_sibling('td').get_text()

    dob_elem = soup.find('td', text='Date of Birth: ')
    dob = dob_elem.find_next_sibling('td').get_text()

    gender_elem = soup.find('td', text='Gender:')
    gender = gender_elem.find_next_sibling('td').get_text()

    race_elem = soup.find('td', text='Race/Ethnicity:')
    race = race_elem.find_next_sibling('td').get_text()

    fac_elem = soup.find('td', text='Location(s) and location phone number(s):')
    fac = fac_elem.find_next_sibling('td').get_text()

    offenses = []

    offense_elem = soup.find_all('td', text='Offense (Class):')
    for offense in offense_elem:
        offenses.append(offense.find_next_sibling('td').get_text())
        
    sentences = []
    sentence_elem = soup.find_all('td', text='Sentence:')
    for sentence in sentence_elem:
        sentences.append(sentence.find_next_sibling('td').get_text())

    offense_sentence_pairs = [list(pair) for pair in zip(offenses, sentences)]

    rows = []
    for pair in offense_sentence_pairs:
        # Create a new row with the offender's information and the current offense and sentence pair
        row = {
            'ID': id,
            'Name': name,
            'Date Of Birth': dob,
            'Age': age,
            'Gender': gender,
            'Race': race,
            'Facility': fac,
            'Offense Description': pair[0],
            'Sentence Length': pair[1]
        }
        rows.append(row)
        # add the new row to the dataframe
    me = pd.concat([me, pd.DataFrame(rows)])

In [ ]:
me.to_csv('/Users/tumendemberelshalkhaan/Desktop/Maine_Raw.csv', index=False)